# TP2 - Gradient Descent

## Introducción

El objetivo de este trabajo es entender mejor gradient descent, basandome en un set de datos reales y generando el algoritmo estudiado de una manera rústica y facil de entender.

## Análisis de los datos

Fuente de datos: https://www.kaggle.com/datasets/bonniesindelar/comparing-progress-of-olympic-winning-track-times

In [38]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # esto hace que las warnings que da pandas al usar su codigo no aparezca
import pandas as pd
import plotly.express as px

In [9]:
df_original = pd.read_csv('Data/women200.csv')

In [10]:
df_original.describe()

,Year,Result,Avg time,"""Change"" value"
count,18.000000,18.000000,1.00,1.0
mean,1983.833333,22.488333,22.49,14.1
std,23.233470,0.853321,NaN,NaN
min,1948.000000,21.530000,22.49,14.1
25%,1965.000000,21.827500,22.49,14.1
50%,1982.000000,22.195000,22.49,14.1
75%,2003.000000,22.875000,22.49,14.1
max,2021.000000,24.400000,22.49,14.1


Solo me interesa quedarme con el año y los tiempos.

In [11]:
df_original= df_original[['Year','Result']].dropna()
df_original.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 0 to 17
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    18 non-null     float64
 1   Result  18 non-null     float64
dtypes: float64(2)
memory usage: 432.0 bytes


### visualización de los datos

In [12]:
px.scatter(df_original, x=df_original['Year'], y=df_original['Result'])

### Normalización

Como la relación de tamaños entre los años y los tiempos es muy grande vamos a normalizar nuestra feature para simplificar las cosas

In [13]:
from sklearn.preprocessing import MinMaxScaler
# Completar el código aquí
scaler = MinMaxScaler()
df_Normalizado = scaler.fit_transform(df_original.iloc[:,:1])
df_Normalizado = pd.DataFrame(df_Normalizado).rename(columns={0:'Año'})
df_Normalizado['Resultado'] = df_original['Result']
df_Normalizado

,Año,Resultado
0,0.000000,24.40
1,0.054795,23.70
2,0.109589,23.40
3,0.164384,24.00
4,0.219178,23.00
5,0.273973,22.50
6,0.328767,22.40
7,0.383562,22.37
8,0.438356,22.03
9,0.493151,21.81


# Hipotesis

El modelo de ML va a ser una regresión lineal común 
$$ h(x)=w_0 + w_1 x$$

In [14]:
def h(x, w0, w1):
  return w0 + w1 * x

Hay que tener en cuenta que la fución es muy rústica y no realiza ninguna validación de datos, por eso al utilizarla tenemos que pasarle los datos de manera correcta. Pero nos dá algo de flexibilidad también, ya que si le pasamos $x$ como un valor númerico nos va a devolver otro valor númerico que sería nuestra predicción de $y$ y si le pasamos $x$ como un np.array o una serie de pandas nos va a devolver el mismo formato con cada valor de $y$ correspondiente

## Función de costo

Para saber si nuestros parámetros w0 y w1 son los óptimos para nuestro modelo tenemos que primero definir la función de costo J, voy a utilizar la siguiente:
$$ J = \frac{1}{2m}  \displaystyle\sum_{i=1}^{m} [ h(x_{i}) - y_{i}]²  $$

In [15]:
# crear función de costo aquí
# x será el conjunto de valores de la feature "Year"
x = df_Normalizado['Año']
# y será el conjunto de valores del target "Result"
y = df_Normalizado['Resultado']
# la función debe retornar un valor numérico

def J(x, y, w0, w1):
  return((((h(x,w0,w1)-y)**2).sum())/(2*df_Normalizado.shape[0]))

In [16]:
J(x,y,0,8)

177.44075901200972

# Gradient Descent

Voy a usar el algoritmo del decenso del gradiente estudiado para encontrar los valores de w0 y w1 que obtengan el valor mínimo en la función de costo J

## Vector Gradiente

Primero me vendría bien una función que me calcule el vector gradiente en un punto cualquiera A(w0,w1) de la función de costo utilizada
$$ \nabla J= [\frac{\partial J}{\partial w_0} ,  \frac{\partial J}{\partial w_1} ] $$

$$ \nabla J = [\frac{1}{m}  \displaystyle\sum_{i=1}^{m}  [h(x_{i}) - y_{i}]  , \frac{1}{m}  \displaystyle\sum_{i=1}^{m}  [h(x_{i}) - y_{i}] x_i ]$$   


In [25]:
def gradient(x, y, w0, w1):
  cord1 = ((h(x,w0,w1)-y).sum())/(df_Normalizado.shape[0])
  cord2 = (((h(x,w0,w1)-y)*x).sum())/(df_Normalizado.shape[0])
  return cord1,cord2

In [27]:
gradient(x, y, 1,1)[0]

-20.997465753424656

## Algoritmo

Muy bien, ahora tengo todo listo para entrenar al modelo, realizo el algoritmo de gradient descent. Comienzo con valores arbitrarios de w0=0 w1=0 y alpha=0.01

tengo que realizar una iteración de n veces arbitrarias (pruebo con valores chicos 10, 20) y en cada una de ellas modificara un poco el punto A(w0, w1) para que vayan en la dirección opuesta al crecimiento de la función J.
$$A_n = A_{n-1} - \alpha  \nabla J(A_{n-1}) $$ 

In [39]:
# La función debe devolver un dataframe que contenga los registros ordenados de 
# cada paso realizado con el valor de w0, w1 y el valor de J
def GradientDescent (x, y, alpha=0.1, steps=10):
  w0 = 0
  w1 = 0
  j = J(x, y, w0, w1)
  Result =  pd.DataFrame(data=[{'w0':w0,'w1':w1,'J':j}])
  for times in range(steps): 
    G = gradient(x,y,w0,w1)
    j = J(x, y, w0, w1)
    w0 = w0 - alpha*G[0]
    w1 = w1 - alpha*G[1]
    data=[{'w0':w0,'w1':w1,'J':j}]
    Result = Result.append(data, ignore_index=True)
  return Result

In [40]:
GradientDescent(x,y,alpha=.5,steps=70).tail()

,w0,w1,J
66,23.145936,-1.413802,0.129990
67,23.164130,-1.448054,0.126801
68,23.181633,-1.481006,0.123850
69,23.198472,-1.512707,0.121118
70,23.214672,-1.543206,0.118590


## Gráfico

¿Cómo se vería nuestro gradiente descendiendo por nuestra función J? 

Con los datos obtenidos del algoritmo de GradientDescent colocaré cada punto obtenido en el plano w0,w1 y los uniré con una linea.

¿Qué pasaria si ajusto el alpha o la cantidad de iteraciones?

In [41]:
df_GD2 = GradientDescent(x,y,alpha=.1,steps=250)
df_GD2

,w0,w1,J
0,0.000000,0.000000,253.206419
1,2.248833,1.081718,253.206419
2,4.219685,2.016635,194.852427
3,5.947560,2.823338,150.262855
4,7.463049,3.518071,116.186156
...,...,...,...
246,22.679861,-0.536359,0.243687
247,22.687036,-0.549867,0.241320
248,22.694157,-0.563273,0.238989
249,22.701224,-0.576577,0.236694


In [42]:
px.scatter(df_GD2, x=df_GD2['w0'], y= df_GD2['w1'])

¿Cómo se vería nuestro modelo en el gráfico de puntos que realizamos en un comienzo? 

In [43]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
df_GD = GradientDescent(x,y,alpha=.5,steps=70)
index = df_GD['J'].idxmin()
w0 = df_GD['w0'].iloc[index]
w1 = df_GD['w1'].iloc[index]
df_YPredicha = pd.DataFrame(h(x, w0, w1)).rename(columns={ 'Año':'Resultado'})
df_YPredicha['year'] = df_original['Year'] 
df_YPredicha

,Resultado,year
0,23.214672,1948.0
1,23.130113,1952.0
2,23.045554,1956.0
3,22.960994,1960.0
4,22.876435,1964.0
5,22.791876,1968.0
6,22.707317,1972.0
7,22.622757,1976.0
8,22.538198,1980.0
9,22.453639,1984.0


In [45]:
import plotly.express as px
import plotly.graph_objects as go
fig1 = px.scatter(df_YPredicha, x=df_original['Year'], y=df_original['Result'])
fig2 = px.line(df_YPredicha,x=df_YPredicha['year'], y=df_YPredicha['Resultado'])
fig3 = go.Figure(data=fig1.data + fig2.data)
fig3.show()